# Using the NIDN with FDTD simulations

This notebook gives explanatory examples on how to use NIDN with FDTD simulations. For more on how FDTD simulations work, see Running_FDTD notebook or the FDTD section in the NIDN docs. 

We start by importing the nidn, or if the nidn is not installed append the root folder to get acces to nidn 

In [ ]:
# Append root folder in case you haven't installed NIDN
import sys
sys.path.append("../")

import nidn

# Inverse Design of a uniform Titanium Dioxide Layer

Here, we define the target spectrum as an already calculated spectrum from a uniform TiO2 layer, and test if NIDN is able to reconstruct the permittivity of the TiO2 layer.

In [ ]:
# Load default cfg as starting point

cfg = nidn.load_default_cfg()

# Specify your desired range of wavelengths

cfg.physical_wavelength_range[0] = 3e-7
cfg.physical_wavelength_range[1] = 1e-6

# Let's investigate 12 frequency points
cfg.N_freq = 12

# Currently, the target spectra is set manually as a list of numbers 
cfg.target_reflectance_spectrum = [0.27891510,0.23644109,0.15938656,0.13485510,0.17970238,0.29539180,0.24678705,0.28984702,0.34120250,0.50414005,0.65206676,0.64914470]
cfg.target_transmittance_spectrum = [0.69567667,0.68589939,0.66614408,0.64175525,0.61354260,0.56891946,0.51561534,0.41896730,0.21201846,0.00011344,0.00000001,0.00000000]
# Since R + T + A = 1, we only need to give the reflectance and transmittance (absorptance is implicit)

nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)

physical_wls, normalized_freqs = nidn.get_frequency_points(cfg)
print("Physical wavelengths are (in meters):")
print(physical_wls)

## Example 1 - Uniform single-layer with restricted epsilon

Let's start with a uniform single-layer and see if NIDN can get sufficiently close to the ground truth.

In [ ]:
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 1 # Choose number of layers

# Allowed range of epsilon values
cfg.real_min_eps = 0.0
cfg.real_max_eps = 20.0
cfg.imag_min_eps = 0.0
cfg.imag_max_eps = 7.0

# Choose model type, regresion or classification
cfg.type = "regression" 
# Set number of training iterations (that is forward model evaluations) to perform
cfg.iterations = 50 

cfg.FDTD_niter = 400
cfg.solver = "FDTD"

In [ ]:
#Show all used settings
nidn.print_cfg(cfg)

`print_cfg(cfg)` shows you more or less everything you want to know about the config.
Using `run_training(cfg)`, we run the network until it reaches the number of iterations set above (or until you interrupt it).

In [ ]:
nidn.run_training(cfg);

### Interpretation of results

#### Loss plot

The loss as a function of model evaluations is presented below. As the training evolves, the three losses here, [L1](https://afteracademy.com/blog/what-are-l1-and-l2-loss-functions), Loss, and Weighted Average Loss, can be seen to decrease. For this regression case, the L1 and Loss are the same.

In [ ]:
nidn.plot_losses(cfg)

#### Spectrum plots

The produced RTA spectra are plotted together with the target spectra in the figure below.

In [ ]:
nidn.plot_spectra(cfg)

#### Absolute grid values plot

The complex absolute value of the epsilon over all frequencies is presented here. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_model_grid(cfg)

#### Epsilon vs frequency and real materials

The following function plots the epsilon values vs. frequency of grid points against real materials in our library. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_eps_per_point(cfg)

## Example 2 - Uniform single-layer with materials classification

Next up is the same example, a uniform single-layer of titanium oxide, but this time we check if NIDN can predict the correct material.

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 1 # Choose number of layers

cfg.type = "classification" # Choose type as described above
cfg.iterations = 20 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

As can be seen from the plots, the prediction is correct and the loss is even lower.

# Classification of three uniform layers of TiO2/GaAs/SiN

This examples shows how NIDN learns the material composition of three uniform layers, with a spectrum previously calculated target spectrum from a simulation with three uniform layers, all 1 um thick, where the first layer is TiO2, the second is GaAs and the third layer is SiN. Note since the total layer thickness is thicker, the number of time-steps is also increased, which makes each iteration in the neural network take more time



In [ ]:
cfg = nidn.load_default_cfg()
# Set the number of frequencies to simulate for
cfg.N_freq = 12
#Number of layers with materials
cfg.N_layers = 3

# Set layer size  to 1x1 (interpreted as uniform)
cfg.Nx = 1 
cfg.Ny = 1
 # Forget the old model
cfg.pop("model",None);
cfg.type = "classification" # Choose type as described above
# Define the thickness of each layer, in default units
cfg.PER_LAYER_THICKNESS=[1.0, 1.0, 1.0]
#Smallest wavelength
cfg.physical_wavelength_range[0]=3e-7
#Largest wavelength
cfg.physical_wavelength_range[1]=15e-7
#init epsiln values
#Convert wavelengths to normalized frequencies used by the layer builder
cfg.target_frequencies = nidn.compute_target_frequencies(
    cfg.physical_wavelength_range[0],
    cfg.physical_wavelength_range[1],
    cfg.N_freq,
    cfg.freq_distribution,
)
# Choose FDTD as simulation type
cfg.solver = "FDTD"
# Number of iterations has to be increased due to a thicker material, thus a further distance for the wave to travel
cfg.FDTD_niter = 800
# Choose pulse and source type
cfg.FDTD_pulse_type = 'continuous'
cfg.FDTD_source_type = 'line'

# Set target spectrum, generated from a simulation with TiO2/GaAs/SiN triple uniform layer
cfg.target_transmittance_spectrum = [0.59790906,0.56570349,0.23946883,0.02939799,0.00184458,0.00002115,0.00000011,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000]
cfg.target_reflectance_spectrum = [0.27799517,0.22808249,0.09394473,0.14923679,0.36340323,0.09872345,0.23317323,0.50220509,0.62269236,0.42524770,0.65769614,0.64963481]
#Number of forward passes in NIDN
cfg.iterations = 50

In [ ]:
# Run the NIDN
nidn.run_training(cfg);

In [ ]:
# Plot the losses, the best spectrum and the epsilon functions for each layer
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_eps_per_point(cfg)